In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=False)

In [ ]:
import sys

sys.path.insert(0, "python")
from experiments import extract_config_columns, prepare_results_for_visualization

In [ ]:
def filter_estimator(
    df: pd.DataFrame, value: str, col: str = "estimator"
) -> pd.DataFrame:
    return df[df[col] == value].copy()

def load_results(fname, only_first_sent=False):
    df = pd.read_csv(fname)
    if "goerli" in fname:
        df["graph_model"] = "goerli_testnet"
    else:
        df["graph_model"] = "random_regular"
    if "active_adversary" in fname:
        df["adversary_type"] = "active"
    else:
        df["adversary_type"] = "passive"
    if "degree" in fname:
        df["adversary_centrality"] = "degree"
    else:
        df["adversary_centrality"] = "none"
    if "bc_all" in fname:
        df["broadcast_mode"] = "all"
    else:
        df["broadcast_mode"] = "sqrt"
    print(df.shape)
    #shorten protocol names before visualization
    df = extract_config_columns(df)
    if only_first_sent:
        df = filter_estimator(df, "first_sent")
    print(df.shape)
    return df


def update_figure(fig, width=1300, height=1000):

    fig.update_layout(
        # TODO: it does not work in case of facet_col.. fix it!
        #    xaxis = dict(
        #        tickmode = 'linear',
        #        tick0 = 0.0,
        #        dtick = 0.05
        #    ),
        width=width,
        height=height,
    )
    fig.update_layout(
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0.0)
    )

In [ ]:
experiment_dir = "scripts/second_round_results/"
rewrite_fig = True#False

# 1. Results with passive adversary

In [ ]:
passive_random_df = load_results("%s/random_regular_1000_none_with_dandelions.csv" % experiment_dir)

In [ ]:
passive_goerli_df = load_results("%s/goerli_none_with_dandelions.csv" % experiment_dir, only_first_sent=True)

## i.) Estimator comparison: first reach vs. first sent

- random regular graph with 1000 nodes and 50 degree
- 5% of all nodes send messages (~ number of messages)

In [ ]:
passive_random_results = prepare_results_for_visualization(passive_random_df)

### Observations (only sanity check)

- Adversary is more efficient with higher adversary ratio - **OK**
- Adversary can better deanonymize simple Broadcast than Dandelion(++) - **OK**
- Dandelion and Dandelion++ has comparable results with the same broadcast probability - **OK**
- Average message spread ratio (fraction of nodes receiving each message) is 1.0 despite the protocol - **OK**

In [ ]:
fig = px.box(
    passive_random_results,
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="estimator",
    facet_row="metric",
)
update_figure(fig)
if rewrite_fig:
    fig.write_image("figures/passive_estimator_check.png")
fig.show()

# NOTE: from now on only first sent estimator results are shown!

## ii.) Graph model comparison: random regular vs. Goerli testnet

- Goerli testnet has approximately 1,5K nodes and 20K edges
- 10% of all nodes send messages (~ number of messages)

In [ ]:
passive_goerli_results = prepare_results_for_visualization(passive_goerli_df)

In [ ]:
passive_first_sent_results = pd.concat(
    [
        filter_estimator(passive_random_results, "first_sent"),
        passive_goerli_results,
    ]
)

### a.) Observations with random adversary sampling (meaningful results)

- Adversary ratio and protocol parameters trend are similar - **OK**
- In general, adversary is less effective for the Goerli testnet than for random regular graphs - **Interesting**
- Dandelion results have smaller deviation (because we use the same network structure 10 times) - **OK**
- Average message spread ratio (fraction of nodes receiving each message) is 1.0 for random regular **BUT** ~0.92 for goerli - **is it OK?** I think yes. Later we show that with broadcasting to all neighbors in the goerli network 1.0 message spread can be achieved

In [ ]:
fig = px.box(
    passive_first_sent_results,
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="graph_model",
    facet_row="metric",
)
update_figure(fig)
if rewrite_fig:
    fig.write_image("figures/graph_model_comparision.png")
fig.show()

### b.) Observations with central node selection (meaningful results)

In [ ]:
passive_random_degree_df = load_results("%s/random_regular_1000_degree_with_dandelions.csv" % experiment_dir, only_first_sent=True)

In [ ]:
passive_goerli_degree_df = load_results("%s/goerli_degree_with_dandelions.csv" % experiment_dir, only_first_sent=True)

In [ ]:
passive_random_degree_results = prepare_results_for_visualization(passive_random_degree_df)
passive_goerli_degree_results = prepare_results_for_visualization(passive_goerli_degree_df)

In [ ]:
passive_degree_results = pd.concat(
    [
        filter_estimator(passive_random_results, "first_sent"),
        passive_goerli_results,
        passive_random_degree_results,
        passive_goerli_degree_results
    ]
)

In [ ]:
fig = px.box(
    passive_degree_results[
        passive_degree_results["metric"] == "hit_ratio"
    ],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="adversary_centrality",
    facet_row="graph_model",
    labels={"value": "hit_ratio"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/passive_adversary_centrality_hit_ratio.png")
fig.show()

In [ ]:
fig = px.box(
    passive_degree_results[
        passive_degree_results["metric"] == "ndcg"
    ],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="adversary_centrality",
    facet_row="graph_model",
    labels={"value": "ndcg"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/passive_adversary_centrality_ndcg.png")
fig.show()

## iii.) Broadcast to all nodes (instead of sqrt amount of them)

In [ ]:
passive_random_bc_all_df = load_results("%s/random_regular_1000_none_with_dandelions_bc_all.csv" % experiment_dir, only_first_sent=True)

In [ ]:
passive_goerli_bc_all_df = load_results("%s/goerli_none_with_dandelions_bc_all.csv" % experiment_dir, only_first_sent=True)

In [ ]:
passive_random_bc_all_results = prepare_results_for_visualization(passive_random_bc_all_df)
passive_goerli_bc_all_results = prepare_results_for_visualization(passive_goerli_bc_all_df)

In [ ]:
passive_broadcast_results = pd.concat(
    [
        filter_estimator(passive_random_results, "first_sent"),
        passive_goerli_results,
        passive_random_bc_all_results,
        passive_goerli_bc_all_results
    ]
)

### Observations (meaningful results)

- Adversary can better deanonymize if message is sent to all neighbors - **OK**

In [ ]:
fig = px.box(
    passive_broadcast_results[
        passive_broadcast_results["metric"] == "hit_ratio"
    ],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="broadcast_mode",
    facet_row="graph_model",
    labels={"value": "hit_ratio"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/broadcast_mode_hit_ratio.png")
fig.show()

- message spread for goerli is less in case of broadcasting to sqrt neighbors - **OK**

In [ ]:
fig = px.box(
    passive_broadcast_results[
        passive_broadcast_results["metric"] == "message_spread_ratio"
    ],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="broadcast_mode",
    facet_row="graph_model",
    labels={"value": "message_spread_ratio"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/broadcast_mode_message_spread.png")
fig.show()

# 2. Active vs. passive adversary

In [ ]:
active_random_df = load_results("%s/random_regular_1000_none_with_dandelions_active_adversary.csv" % experiment_dir, only_first_sent=True)
active_random_degree_df = load_results("%s/random_regular_1000_degree_with_dandelions_active_adversary.csv" % experiment_dir, only_first_sent=True)

In [ ]:
active_goerli_df = load_results("%s/goerli_none_with_dandelions_active_adversary.csv" % experiment_dir, only_first_sent=True)
active_goerli_degree_df = load_results("%s/goerli_degree_with_dandelions_active_adversary.csv" % experiment_dir, only_first_sent=True)

In [ ]:
active_random_results = prepare_results_for_visualization(active_random_df)
active_goerli_results = prepare_results_for_visualization(active_goerli_df)
active_random_degree_results = prepare_results_for_visualization(active_random_degree_df)
active_goerli_degree_results = prepare_results_for_visualization(active_goerli_degree_df)

In [ ]:
passive_active_results = pd.concat(
    [
        filter_estimator(passive_random_results, "first_sent"),
        passive_goerli_results,
        active_random_results,
        active_goerli_results
    ]
)

In [ ]:
passive_active_degree_results = pd.concat(
    [
        active_random_results,
        active_goerli_results,
        active_random_degree_results,
        active_goerli_degree_results
    ]
)

### Observations (meaningful)

- Active adversary effect on message spread behaves as expected - **OK**

In [ ]:
fig = px.box(
    passive_active_results[passive_active_results["metric"] == "message_spread_ratio"],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="adversary_type",
    facet_row="graph_model",
    labels={"value": "message_spread_ratio"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/passive_vs_active_adversary_message_spread.png")
fig.show()

* and it is even worse if adversaries are high centrality nodes

In [ ]:
fig = px.box(
    passive_active_degree_results[passive_active_degree_results["metric"] == "message_spread_ratio"],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="adversary_centrality",
    facet_row="graph_model",
    labels={"value": "message_spread_ratio"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/passive_vs_active_adversary_centrality_message_spread.png")
fig.show()

- There is no significant difference in adversary performance **(as expected)**

In [ ]:
fig = px.box(
    passive_active_results[passive_active_results["metric"] == "ndcg"],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="adversary_type",
    facet_row="graph_model",
    labels={"value": "ndcg"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/passive_vs_active_adversary_ndcg.png")
fig.show()

In [ ]:
fig = px.box(
    passive_active_results[passive_active_results["metric"] == "hit_ratio"],
    x="adversary_ratio",
    y="value",
    color="protocol",
    facet_col="adversary_type",
    facet_row="graph_model",
    labels={"value": "hit_ratio"},
)
update_figure(fig, 1200, 500)
if rewrite_fig:
    fig.write_image("figures/passive_vs_active_adversary_hit_ratio.png")
fig.show()

### TODO: experiment with random regular degree (10K nodes)
### TODO: experiment with weighted nodes: stake (for message source selection)

# Node contact time cexperiments

In [ ]:
def extract_contact_time_quantiles(
    dataframe, group_cols=["protocol"], target_col="mean_contact_time_quantiles"
):
    df = dataframe.copy()
    df[target_col] = df[target_col].apply(eval)
    mean_time_q_aggr = (
        filter_estimator(df, "first_sent")
        .groupby(group_cols)[target_col]
        .sum()
        .reset_index()
    )
    # print(mean_time_q_aggr.head())
    mean_time_q_aggr[target_col] = mean_time_q_aggr[target_col].apply(
        lambda x: np.mean(np.array(x).reshape(-1, 9), axis=0)
    )
    old_cols = set(mean_time_q_aggr.columns)
    mean_time_q_aggr = mean_time_q_aggr.assign(
        **pd.DataFrame(mean_time_q_aggr[target_col].values.tolist()).add_prefix("q_")
    )
    diff_cols = sorted(list(set(mean_time_q_aggr.columns).difference(old_cols)))
    visu_df = mean_time_q_aggr.melt(
        id_vars=group_cols, value_vars=diff_cols, value_name="time", var_name="quantile"
    )
    replace_map = dict(zip(diff_cols, np.arange(0.1, 1.1, 0.1)))
    visu_df["quantile"] = visu_df["quantile"].replace(replace_map)
    return visu_df

### Nodes are reached slower on the Goerli testnet than on random regular graph - **OK** (Goerli is larger)

In [ ]:
visu1 = extract_contact_time_quantiles(passive_random_df)
visu1["graph_model"] = "random_regular"
visu2 = extract_contact_time_quantiles(passive_goerli_df)
visu2["graph_model"] = "goerli_testnet"
visu_df = pd.concat([visu1, visu2])

In [ ]:
fig = px.line(visu_df, x="quantile", y="time", color="protocol", symbol="graph_model")
fig.show()

### Nodes are reached slower in case of "sqrt" boradcast setting - **OK**

In [ ]:
visu1 = extract_contact_time_quantiles(passive_random_bc_all_df)
visu1["broadcast_mode"] = "all"
visu2 = extract_contact_time_quantiles(passive_random_df)
visu2["broadcast_mode"] = "sqrt"
visu_df = pd.concat([visu1, visu2])

In [ ]:
fig = px.line(
    visu_df, x="quantile", y="time", color="protocol", symbol="broadcast_mode"
)
fig.show()

### TODO: use different quantiles: 0.5%, 1%, 5%, 10%, 50%, 90% 